## Making links more reusable

Each time a string is encountered that is a link assign the link in the `globals` namespace.  The links are not valid variable names so they will be difficult to misuse.  The IPython tab completion with return the link names.

    %reload_ext deathbeds.__Links_in_working_namespace

In [147]:
import ast, IPython, CommonMark; ip = IPython.get_ipython()

Each string that is a link will be assigned to the interactive `IPython` namespace.

In [149]:
class GlobalLinks(ast.NodeTransformer):
    def visit_Str(self, node):
        if 'http' in node.s:
            for link in node.s.split('http')[1:]:
                link = ('http'+link).split()[0].rstrip(')')
                __import__('IPython').get_ipython().user_ns[link] = link                
        LinkExtract().render(CommonMark.Parser().parse(node.s))
        return node

We can add methods that introspect the Markdown to discover deeper linking.

In [150]:
class LinkExtract(CommonMark.render.renderer.Renderer):
    def link(self, node, entering):
        if entering:
            ns = __import__('IPython').get_ipython().user_ns
            ns[node.destination] = node.destination
            
            if node.title:
                ns[node.title] = node.destination

Load the extensions.

In [151]:
def load_ipython_extension(ip):
    ip.ast_transformers = [GlobalLinks()] + (unload_ipython_extension(ip) or ip.ast_transformers)
    
def unload_ipython_extension(ip):
     ip.ast_transformers = [object for object in ip.ast_transformers if not isinstance(object, GlobalLinks)]

__name__ == '__main__' and load_ipython_extension(get_ipython())